In [42]:
import pandas as pd
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.models import Sequential,load_model
from keras.layers import LSTM, Dense,Dropout,Bidirectional
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
tf.get_logger().setLevel('ERROR')

In [43]:
df = pd.read_csv('../dataset/vms_scheduling_dataset_2000.csv') 
unique_labels = np.unique(df['VmID'])
num_classes=len(unique_labels)
print(unique_labels)
print(df.info())


[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TaskID              2000 non-null   int64  
 1   TaskFileSize        2000 non-null   int64  
 2   TaskOutputFileSize  2000 non-null   int64  
 3   TaskFileLength      2000 non-null   int64  
 4   UserLatitude        2000 non-null   float64
 5   UserLongitude       2000 non-null   float64
 6   VmProcessingSpeed   2000 non-null   float64
 7   VmRam               2000 non-null   int64  
 8   VmBandwidth         2000 non-null   int64  
 9   VmStorage           2000 non-null   int64  
 10  VmMemoryCost        2000 non-null   float64
 11  VmStorageCost       2000 non-null   float64
 12  VmBandwidthCost     2000 non-null   float64
 13  VmProcessingCost    2000 non-null   float64
 14  CpuTime             2000 non-null   float64
 15  TotalLength

In [44]:
task_columns=['CostPerSec', 'StartExecTime','SerivesLevel']
location_columns=['UserLatitude', 'UserLongitude']
vms_columns=['VmProcessingSpeed','VmRam','VmBandwidth','VmStorage','VmMemoryCost','VmStorageCost','VmBandwidthCost','VmProcessingCost']

df.drop(columns=vms_columns+task_columns+location_columns, inplace=True)

X = df.drop('VmID', axis=1)  # Features
y=df['VmID']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(np.unique(y))
print(df.columns)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Index(['TaskID', 'TaskFileSize', 'TaskOutputFileSize', 'TaskFileLength',
       'CpuTime', 'TotalLength', 'VmID'],
      dtype='object')


In [45]:
if (len(X_train.shape)<3):
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(" xtrain ", X_train.shape, "\n", "ytrain ", y_train.shape, "\n", "xtest  ", X_test.shape, "\n", "ytest ", y_test.shape, "\n")




 xtrain  (1600, 1, 6) 
 ytrain  (1600,) 
 xtest   (400, 1, 6) 
 ytest  (400,) 



In [ ]:
def trainModel(hyperparameters):
    # Build and train the model
    model = Sequential()
    model.add(LSTM(units=int(hyperparameters['units']), input_shape=(1, X_train.shape[2]), return_sequences=True))
    model.add(Dropout(hyperparameters['dropout_rate']))
    model.add(LSTM(units=int(hyperparameters['units']) // 2))
    model.add(Dense(units=num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hyperparameters['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    hist = model.fit(X_train, y_train,
                        epochs=100,
                        batch_size=int(hyperparameters['batch_size']),
                        validation_data=(X_test, y_test),
                       )
    return model,hist

SNAKE_hyperparameters = {'units': 128, 'dropout_rate': 0.01, 'learning_rate': 0.001, 'batch_size': 32}

snake_model,snake_hist= trainModel(Best_Hyperparameters)


Epoch 1/100
 1/50 [..............................] - ETA: 3:18 - loss: 2.6572 - accuracy: 0.0000e+00

In [ ]:
snake_accuracy = max(snake_hist.history["accuracy"])

print("snake  Accuracy: ",np.round(snake_accuracy* 100) ,'%')


In [48]:
# snake_model.save('../models/snake_vms_scheduling_86.keras')